In [22]:
#base -- original

import cv2
import tkinter as tk
import os
import numpy as np
import time

### idhi github la kelli eskochina already pre-trained model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)


cam = cv2.VideoCapture(0)


cam.set(cv2.CAP_PROP_FRAME_WIDTH, 520)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 380)
cam.set(cv2.CAP_PROP_FPS, 30)

dress_images = []
current_dress_index = 0
user_picture_path = r'.\assets'

################################ kindha code gpt la kelli eskochindhi ###################################

def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(12):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        #print('Selected shirt image:', image_path) #idhi anvasaram ga print chesthandhi errors appudu enable cheskundham

        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')

def process_dress_image(img): ## dheeni baadha endho kuda ardham aiethaledh, okka transparency osthey aiepothadhi ga saava dobbhuthandhi
    try:
        if img.shape[2] == 4:
            return img  # Image already has an alpha channel
        else:
            # Split channels
            b, g, r = cv2.split(img)
            
            # Create alpha channel (fully opaque)
            alpha = np.ones_like(b) * 255
            
            # Merge channels to form RGBA image
            rgba_img = cv2.merge((b, g, r, alpha))
            
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

    
################################ paina code gpt la kelli eskochindhi but waste ###################################

def apply_shirt(frame, dress_index, detections):
    global dress_images
    if dress_images and dress_index < len(dress_images):
        confidence_threshold = 0.5  
        detected_faces = []
        
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            
            if confidence > confidence_threshold:
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                detected_faces.append((x, y, w, h))

        if detected_faces:
            dress_img = dress_images[dress_index]
            aspect_ratio = dress_img.shape[1] / dress_img.shape[0]

            for (x, y, w, h) in detected_faces:
                y_shirt = y + int(0.8 * h)
                h_shirt = int(0.5 * h)
                x_offset = int(0.5 * w)
                x_shirt = max(x - x_offset, 0)
                w_shirt = min(w + 2 * x_offset, frame.shape[1] - x_shirt)
                new_h_shirt = int(w_shirt / aspect_ratio)

                if new_h_shirt > 0:
                    resized_shirt = cv2.resize(dress_img, (w_shirt, new_h_shirt), interpolation=cv2.INTER_LINEAR)

                    if resized_shirt.shape[2] == 4:
                        resized_shirt = resized_shirt[:, :, :3]

                    frame[y_shirt:y_shirt + new_h_shirt, x_shirt:x_shirt + w_shirt] = resized_shirt[
                                                                                        :min(new_h_shirt,
                                                                                             frame.shape[0] - y_shirt),
                                                                                        :min(w_shirt,
                                                                                             frame.shape[1] - x_shirt)]
                    
##########################   ee paina code gpt improvise chesindhi changes em cheyyodhu malla pisukuthadhi   ##########################
                    
def process_and_save_output(frame, dress_index, detections):
    global user_picture_path
    if dress_images and user_picture_path:
        output_folder = 'outputs'
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, f'output{dress_index}.png')  
        frame_copy = frame.copy()
        apply_shirt(frame_copy, dress_index, detections)
        cv2.imwrite(output_path, frame_copy)
        print(f'Saved output image: {output_path}')

def main():
    select_dress_images()
    global current_dress_index

    start_time = time.time()
    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        apply_shirt(frame, current_dress_index, detections)
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break
        elif key == 13:
            process_and_save_output(frame, current_dress_index, detections)
            current_dress_index = (current_dress_index + 1) % len(dress_images)
            if current_dress_index == 0: 
                break
        
        if time.time() - start_time >= 1:
            process_and_save_output(frame, current_dress_index, detections)
            current_dress_index = (current_dress_index + 1) % len(dress_images)
            if current_dress_index == 0: 
                break
            start_time = time.time()

    cv2.destroyAllWindows()
    cam.release()

main()


Error loading image: .\assets\shirt11.png
Saved output image: outputs\output0.png
Saved output image: outputs\output1.png
Saved output image: outputs\output2.png
Saved output image: outputs\output3.png
Saved output image: outputs\output4.png
Saved output image: outputs\output5.png
Saved output image: outputs\output6.png
Saved output image: outputs\output7.png
Saved output image: outputs\output8.png
Saved output image: outputs\output9.png
Saved output image: outputs\output10.png


In [28]:
#base -- transparency
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Load dress images with alpha channel
def load_dress_images(folder_path):
    dress_images = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            dress_images.append(img)
        else:
            print(f'Error loading image: {file_path}')
    return dress_images

# Overlay dress on detected faces
def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor, fy=scale_factor)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(0.8 * h)
    dress_x = x + int((w - resized_dress.shape[1]) / 2)

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress mask to match the size of the ROI
    resized_dress_mask = cv2.resize(dress_img[:, :, 3], (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress_mask / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Ensure the dress and ROI have the same shape
    if resized_dress.shape[:2] != roi.shape[:2]:
        resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi


# Main function
def main():
    # Folder path containing dress images
    dress_folder_path = r'.\assets'

    # Load dress images
    dress_images = load_dress_images(dress_folder_path)

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


## producing final code

In [3]:
#this is only saving one
import cv2
import numpy as np
import os
import time

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Load dress images with alpha channel
def load_dress_images(folder_path):
    dress_images = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            dress_images.append(img)
        else:
            print(f'Error loading image: {file_path}')
    return dress_images

# Overlay dress on detected faces
def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor, fy=scale_factor)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(0.8 * h)
    dress_x = x + int((w - resized_dress.shape[1]) / 2)

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress mask to match the size of the ROI
    resized_dress_mask = cv2.resize(dress_img[:, :, 3], (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress_mask / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Ensure the dress and ROI have the same shape
    if resized_dress.shape[:2] != roi.shape[:2]:
        resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi

#saving
def process_and_save_output(frame, detections, dress_index):
    dress_index = 0

    # global user_picture_path
    # user_picture_path = 'outputls'
    # if user_picture_path:
    output_folder = 'outputs'
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, f'output{dress_index}.png')  
    dress_index = dress_index + 1
    frame_copy = frame.copy()

    cv2.imwrite(output_path, frame_copy)
    print(f'Saved output image: {output_path}')

        

# Main function
def main():
    # Folder path containing dress images
    dress_folder_path = r'.\assets'

    dress_index = None
    # Load dress images
    dress_images = load_dress_images(dress_folder_path)

    start_time = time.time()

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:

                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        key = cv2.waitKey(1)
        if key == ord('.'):
            break
        elif key == 13:
            process_and_save_output(frame,  detections, dress_index)
            # if current_dress_index == 0: 
            #     break
        
        if time.time() - start_time >= 1:
            process_and_save_output(frame, detections, dress_index)
            # if current_dress_index == 0: 
            #     break
            # start_time = time.time()

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png
Saved output image: outputs\output0.png


KeyboardInterrupt: 

In [4]:
#only saving output0_0
import cv2
import numpy as np
import os
import time

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variable for dress index
dress_index = 0

# Load dress images with alpha channel
def load_dress_images(folder_path):
    dress_images = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if img is not None:
            dress_images.append(img)
        else:
            print(f'Error loading image: {file_path}')
    return dress_images

# Overlay dress on detected faces
def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor, fy=scale_factor)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(0.8 * h)
    dress_x = x + int((w - resized_dress.shape[1]) / 2)

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress mask to match the size of the ROI
    resized_dress_mask = cv2.resize(dress_img[:, :, 3], (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress_mask / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Ensure the dress and ROI have the same shape
    if resized_dress.shape[:2] != roi.shape[:2]:
        resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi

# Saving images with dress overlays
def process_and_save_output(frame, detections, dress_images):
    global dress_index
    output_folder = 'outputs'
    os.makedirs(output_folder, exist_ok=True)

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Confidence threshold
            # Get face coordinates
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (x, y, w, h) = box.astype(int)

            # Apply dress on detected face
            dress_index = i % len(dress_images)
            dress_img = dress_images[dress_index]
            frame_with_dress = frame.copy()  # Create a copy of the frame to avoid modifying the original frame
            apply_dress(frame_with_dress, dress_img, x, y, w, h)

            # Save output image with a unique filename for each detected face
            output_path = os.path.join(output_folder, f'output_dress_{dress_index}_{i}.png')
            cv2.imwrite(output_path, frame_with_dress)
            print(f'Saved output image: {output_path}')

# Main function
def main():
    # Folder path containing dress images
    dress_folder_path = r'.\assets'

    # Load dress images
    dress_images = load_dress_images(dress_folder_path)

    start_time = time.time()

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Process and save output images
        process_and_save_output(frame, detections, dress_images)

        key = cv2.waitKey(1)
        if key == ord('.'):
            break
        
        if time.time() - start_time >= 1:
            start_time = time.time()  # Reset the start time

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: outputs\output_dress_0_0.png
Saved output image: 

KeyboardInterrupt: 

# produc

In [30]:
#check  point 1
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variables
dress_images = []
user_picture_path = r'.\assets'
total_dresses = 11
# Load dress images
def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(total_dresses):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        print(image_path)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')

# Overlay dress on detected faces
def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor, fy=scale_factor)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(0.8 * h)
    dress_x = x + int((w - resized_dress.shape[1]) / 2)

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress mask to match the size of the ROI
    resized_dress_mask = cv2.resize(dress_img[:, :, 3], (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress_mask / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Ensure the dress and ROI have the same shape
    if resized_dress.shape[:2] != roi.shape[:2]:
        resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi

# Main function
def main():
    # Select dress images
    select_dress_images()

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


.\assets\shirt0.png
.\assets\shirt1.png
.\assets\shirt2.png
.\assets\shirt3.png
.\assets\shirt4.png
.\assets\shirt5.png
.\assets\shirt6.png
.\assets\shirt7.png
.\assets\shirt8.png
.\assets\shirt9.png
.\assets\shirt10.png


In [31]:
#check  point 2
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variables
dress_images = []
user_picture_path = r'.\assets'
total_dresses = 11
# Load dress images
def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(total_dresses):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        print(image_path)
        if img is not None:
            processed_img = process_dress_image(img)
            if processed_img is not None:
                dress_images.append(processed_img)
            else:
                print(f'Error processing image: {image_path}')
        else:
            print(f'Error loading image: {image_path}')
            

def process_dress_image(img): ## dheeni baadha endho kuda ardham aiethaledh, okka transparency osthey aiepothadhi ga saava dobbhuthandhi
    try:
        if img.shape[2] == 4:
            return img  # Image already has an alpha channel
        else:
            # Split channels
            b, g, r = cv2.split(img)
            
            # Create alpha channel (fully opaque)
            alpha = np.ones_like(b) * 255
            
            # Merge channels to form RGBA image
            rgba_img = cv2.merge((b, g, r, alpha))
            
            return rgba_img
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face and enlarge it even more
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor * 2.0, fy=scale_factor * 2.0)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(0.8 * h)

    # Adjust the x-coordinate to move the dress more to the right
    dress_x = x - int((resized_dress.shape[1] - w) / 2) - 300  # Adjust the value as needed

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress mask to match the size of the ROI
    resized_dress_mask = cv2.resize(dress_img[:, :, 3], (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress_mask / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Ensure the dress and ROI have the same shape
    if resized_dress.shape[:2] != roi.shape[:2]:
        resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi



# Main function
def main():
    # Select dress images
    select_dress_images()

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


.\assets\shirt0.png
.\assets\shirt1.png
.\assets\shirt2.png
.\assets\shirt3.png
.\assets\shirt4.png
.\assets\shirt5.png
.\assets\shirt6.png
.\assets\shirt7.png
.\assets\shirt8.png
.\assets\shirt9.png
.\assets\shirt10.png


In [24]:
#check  point 3
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variables
dress_images = []
user_picture_path = r'.\assets'
total_dresses = 11
# Load dress images
def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(total_dresses):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        print(image_path)
        dress_images.append(img)

def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face and enlarge it even more
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor * 2.0, fy=scale_factor * 2.0)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(1.2 * h)

    # Adjust the x-coordinate to move the dress more to the right
    dress_x = x - int((resized_dress.shape[1] - w) / 2) - 250  # Adjust the value as needed

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Get the region of interest in the frame for the dress overlay
    roi = frame[dress_y:dress_y_end, dress_x:dress_x_end]

    # Resize the dress mask to match the size of the ROI
    resized_dress_mask = cv2.resize(dress_img[:, :, 3], (roi.shape[1], roi.shape[0]))

    # Create a mask for the dress overlay
    dress_mask = resized_dress_mask / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Ensure the dress and ROI have the same shape
    if resized_dress.shape[:2] != roi.shape[:2]:
        resized_dress = cv2.resize(resized_dress, (roi.shape[1], roi.shape[0]))

    # Loop through each channel (BGR) and perform element-wise operations
    for c in range(0, 3):
        roi[:, :, c] = (dress_mask * resized_dress[:, :, c] +
                        inv_dress_mask * roi[:, :, c])

    # Update the region of interest in the frame with the dress overlay
    frame[dress_y:dress_y_end, dress_x:dress_x_end] = roi


# Main function
def main():
    # Select dress images
    select_dress_images()

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


.\assets\shirt0.png
.\assets\shirt1.png
.\assets\shirt2.png
.\assets\shirt3.png
.\assets\shirt4.png
.\assets\shirt5.png
.\assets\shirt6.png
.\assets\shirt7.png
.\assets\shirt8.png
.\assets\shirt9.png
.\assets\shirt10.png


# here is important


In [25]:
#check  point 3.0 #left over will not be shown in camera
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variables
dress_images = []
user_picture_path = r'.\assets'
total_dresses = 11
# Load dress images
def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(total_dresses):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        print(image_path)
        dress_images.append(img)

def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face and enlarge it slightly
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor * 2.5, fy=scale_factor * 2.5)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(h * 0.8)  # Adjust the value to position the dress below the neck
    dress_x = x - int((resized_dress.shape[1] - w) / 2) - 270

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Calculate the region of interest for the dress overlay
    dress_roi_y_start = max(dress_y, 0)
    dress_roi_y_end = min(dress_y_end, frame.shape[0])
    dress_roi_x_start = max(dress_x, 0)
    dress_roi_x_end = min(dress_x_end, frame.shape[1])

    # Calculate the region of interest in the dress image
    dress_img_roi_y_start = dress_roi_y_start - dress_y
    dress_img_roi_y_end = dress_img_roi_y_start + (dress_roi_y_end - dress_roi_y_start)
    dress_img_roi_x_start = dress_roi_x_start - dress_x
    dress_img_roi_x_end = dress_img_roi_x_start + (dress_roi_x_end - dress_roi_x_start)

    # Extract the dress overlay and mask within the region of interest
    dress_overlay = resized_dress[dress_img_roi_y_start:dress_img_roi_y_end, dress_img_roi_x_start:dress_img_roi_x_end]
    dress_mask = dress_overlay[:, :, 3] / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Apply the dress overlay to the frame within the region of interest
    frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3] = (
            dress_mask[:, :, np.newaxis] * dress_overlay[:, :, :3] +
            inv_dress_mask[:, :, np.newaxis] * frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3]
    )



# Main function
def main():
    # Select dress images
    select_dress_images()

    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = i % len(dress_images)
                    dress_img = dress_images[dress_index]
                    apply_dress(frame, dress_img, x, y, w, h)

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


.\assets\shirt0.png
.\assets\shirt1.png
.\assets\shirt2.png
.\assets\shirt3.png
.\assets\shirt4.png
.\assets\shirt5.png
.\assets\shirt6.png
.\assets\shirt7.png
.\assets\shirt8.png
.\assets\shirt9.png
.\assets\shirt10.png


In [1]:
#check  point 3.1 #left over will not be shown in camera and also the total transparent
import cv2
import numpy as np
import os

# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variables
dress_images = []

user_picture_path = r'.\assets'
total_dresses = 11
# Load dress images
def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(total_dresses):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        print(image_path)
        dress_images.append(img)

def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face and enlarge it slightly
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor * 2.5, fy=scale_factor * 2.5)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(h * 0.8)  # Adjust the value to position the dress below the neck
    dress_x = x - int((resized_dress.shape[1] - w) / 2) - 250

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Calculate the region of interest for the dress overlay
    dress_roi_y_start = max(dress_y, 0)
    dress_roi_y_end = min(dress_y_end, frame.shape[0])
    dress_roi_x_start = max(dress_x, 0)
    dress_roi_x_end = min(dress_x_end, frame.shape[1])

    # Calculate the region of interest in the dress image
    dress_img_roi_y_start = dress_roi_y_start - dress_y
    dress_img_roi_y_end = dress_img_roi_y_start + (dress_roi_y_end - dress_roi_y_start)
    dress_img_roi_x_start = dress_roi_x_start - dress_x
    dress_img_roi_x_end = dress_img_roi_x_start + (dress_roi_x_end - dress_roi_x_start)

    # Extract the dress overlay and mask within the region of interest
    dress_overlay = resized_dress[dress_img_roi_y_start:dress_img_roi_y_end, dress_img_roi_x_start:dress_img_roi_x_end]
    dress_mask = dress_overlay[:, :, -1] / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Apply the dress overlay to the frame within the region of interest
    frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3] = (
            dress_mask[:, :, np.newaxis] * dress_overlay[:, :, :3] +
            inv_dress_mask[:, :, np.newaxis] * frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3]
    )

def process_and_save_output(frame, dress_index, detections, x, y, w, h):
    global user_picture_path
    if dress_images and user_picture_path:
        output_folder = 'outputs'
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, f'output{dress_index}.png')  
        frame_copy = frame.copy()
        dress_img = dress_images[dress_index]
        apply_dress(frame_copy, dress_img, x, y, w, h)
        cv2.imwrite(output_path, frame_copy)
        print(f'Saved output image: {output_path}')

def main():
    # Select dress images
    select_dress_images()
    
    current_dress_index = 0
    processed_all_dresses = False

    while cam.isOpened() and not processed_all_dresses:
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    dress_index = current_dress_index % len(dress_images)
                    process_and_save_output(frame, dress_index, detections, x, y, w, h)
                    current_dress_index += 1

                    if current_dress_index >= len(dress_images):
                        processed_all_dresses = True
                        break

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


.\assets\shirt0.png
.\assets\shirt1.png
.\assets\shirt2.png
.\assets\shirt3.png
.\assets\shirt4.png
.\assets\shirt5.png
.\assets\shirt6.png
.\assets\shirt7.png
.\assets\shirt8.png
.\assets\shirt9.png
.\assets\shirt10.png
Saved output image: outputs\output0.png
Saved output image: outputs\output1.png
Saved output image: outputs\output2.png
Saved output image: outputs\output3.png
Saved output image: outputs\output4.png
Saved output image: outputs\output5.png
Saved output image: outputs\output6.png
Saved output image: outputs\output7.png
Saved output image: outputs\output8.png
Saved output image: outputs\output9.png
Saved output image: outputs\output10.png


In [34]:
#check  point 3.2 #left over will not be shown in camera and also the total transparent
import cv2
import numpy as np
import os
import time
# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variables
dress_images = []

user_picture_path = r'.\assets'
total_dresses = 11
# Load dress images
def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(total_dresses):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        print(image_path)
        dress_images.append(img)

def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the minimum scale factor to ensure the dress fits within the detected face
    scale_factor = min(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face and enlarge it slightly
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor * 2.5, fy=scale_factor * 2.5)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(h * 0.8)  # Adjust the value to position the dress below the neck
    dress_x = x - int((resized_dress.shape[1] - w) / 2) - 250

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Calculate the region of interest for the dress overlay
    dress_roi_y_start = max(dress_y, 0)
    dress_roi_y_end = min(dress_y_end, frame.shape[0])
    dress_roi_x_start = max(dress_x, 0)
    dress_roi_x_end = min(dress_x_end, frame.shape[1])

    # Calculate the region of interest in the dress image
    dress_img_roi_y_start = dress_roi_y_start - dress_y
    dress_img_roi_y_end = dress_img_roi_y_start + (dress_roi_y_end - dress_roi_y_start)
    dress_img_roi_x_start = dress_roi_x_start - dress_x
    dress_img_roi_x_end = dress_img_roi_x_start + (dress_roi_x_end - dress_roi_x_start)

    # Extract the dress overlay and mask within the region of interest
    dress_overlay = resized_dress[dress_img_roi_y_start:dress_img_roi_y_end, dress_img_roi_x_start:dress_img_roi_x_end]
    dress_mask = dress_overlay[:, :, -1] / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Apply the dress overlay to the frame within the region of interest
    frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3] = (
            dress_mask[:, :, np.newaxis] * dress_overlay[:, :, :3] +
            inv_dress_mask[:, :, np.newaxis] * frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3]
    )

def process_and_save_output(frame, dress_index, detections, x, y, w, h):
    global user_picture_path
    if dress_images and user_picture_path:
        output_folder = 'outputs'
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, f'output{dress_index}.png')  
        frame_copy = frame.copy()
        dress_img = dress_images[dress_index]
        apply_dress(frame_copy, dress_img, x, y, w, h)
        cv2.imwrite(output_path, frame_copy)
        print(f'Saved output image: {output_path}')

def main():
    # Select dress images
    select_dress_images()
    
    start_time = time.time()
    current_dress_index = 0
    processed_all_dresses = False

    while cam.isOpened() and not processed_all_dresses:
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    if time.time() - start_time >= 1:
                        dress_index = current_dress_index % len(dress_images)
                        process_and_save_output(frame, dress_index, detections, x, y, w, h)
                        current_dress_index += 1

                        if current_dress_index >= len(dress_images):
                            processed_all_dresses = True
                            break

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


.\assets\shirt0.png
.\assets\shirt1.png
.\assets\shirt2.png
.\assets\shirt3.png
.\assets\shirt4.png
.\assets\shirt5.png
.\assets\shirt6.png
.\assets\shirt7.png
.\assets\shirt8.png
.\assets\shirt9.png
.\assets\shirt10.png
Saved output image: outputs\output0.png
Saved output image: outputs\output1.png
Saved output image: outputs\output2.png
Saved output image: outputs\output3.png
Saved output image: outputs\output4.png
Saved output image: outputs\output5.png
Saved output image: outputs\output6.png
Saved output image: outputs\output7.png
Saved output image: outputs\output8.png
Saved output image: outputs\output9.png
Saved output image: outputs\output10.png


In [3]:
#check  point 3.3 #left over will not be shown in camera and also the total transparent
import cv2
import numpy as np
import os
import time
# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variables
dress_images = []

user_picture_path = r'.\assets'
total_dresses = 11
# Load dress images
def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(total_dresses):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        print(image_path)
        dress_images.append(img)

def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the maximum scale factor to enlarge the dress as much as possible while fitting within the detected face
    scale_factor = max(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor * 1.5, fy=scale_factor * 1.5)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(h * 0.8)  # Adjust the value to position the dress below the neck
    dress_x = x - int((resized_dress.shape[1] - w) / 2) - 200

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Calculate the region of interest for the dress overlay
    dress_roi_y_start = max(dress_y, 0)
    dress_roi_y_end = min(dress_y_end, frame.shape[0])
    dress_roi_x_start = max(dress_x, 0)
    dress_roi_x_end = min(dress_x_end, frame.shape[1])

    # Calculate the region of interest in the dress image
    dress_img_roi_y_start = dress_roi_y_start - dress_y
    dress_img_roi_y_end = dress_img_roi_y_start + (dress_roi_y_end - dress_roi_y_start)
    dress_img_roi_x_start = dress_roi_x_start - dress_x
    dress_img_roi_x_end = dress_img_roi_x_start + (dress_roi_x_end - dress_roi_x_start)

    # Extract the dress overlay and mask within the region of interest
    dress_overlay = resized_dress[dress_img_roi_y_start:dress_img_roi_y_end, dress_img_roi_x_start:dress_img_roi_x_end]
    dress_mask = dress_overlay[:, :, -1] / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Apply the dress overlay to the frame within the region of interest
    frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3] = (
            dress_mask[:, :, np.newaxis] * dress_overlay[:, :, :3] +
            inv_dress_mask[:, :, np.newaxis] * frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3]
    )



def process_and_save_output(frame, dress_index, detections, x, y, w, h):
    global user_picture_path
    if dress_images and user_picture_path:
        output_folder = 'outputs'
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, f'output{dress_index}.png')  
        frame_copy = frame.copy()
        dress_img = dress_images[dress_index]
        apply_dress(frame_copy, dress_img, x, y, w, h)
        cv2.imwrite(output_path, frame_copy)
        print(f'Saved output image: {output_path}')

def main():
    # Select dress images
    select_dress_images()
    
    start_time = time.time()
    current_dress_index = 0
    processed_all_dresses = False

    while cam.isOpened() and not processed_all_dresses:
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    if time.time() - start_time >= 1:
                        dress_index = current_dress_index % len(dress_images)
                        process_and_save_output(frame, dress_index, detections, x, y, w, h)
                        current_dress_index += 1

                        if current_dress_index >= len(dress_images):
                            processed_all_dresses = True
                            break

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


.\assets\shirt0.png
.\assets\shirt1.png
.\assets\shirt2.png
.\assets\shirt3.png
.\assets\shirt4.png
.\assets\shirt5.png
.\assets\shirt6.png
.\assets\shirt7.png
.\assets\shirt8.png
.\assets\shirt9.png
.\assets\shirt10.png
Saved output image: outputs\output0.png
Saved output image: outputs\output1.png
Saved output image: outputs\output2.png
Saved output image: outputs\output3.png
Saved output image: outputs\output4.png
Saved output image: outputs\output5.png
Saved output image: outputs\output6.png
Saved output image: outputs\output7.png
Saved output image: outputs\output8.png
Saved output image: outputs\output9.png
Saved output image: outputs\output10.png
